# Part 15: Beacon Exclusion Zone

## Setup

In [1]:
data = [line for line in open("./inputs/day15.txt", "r").read().splitlines()]

## Helpers
Line parser that finds indices by '=', and parses the four values.

In [2]:
def parse_line(line):
  indices = [i for i, c in enumerate(line) if c == '=']
  sensor_x, sensor_y, beacon_x, beacon_y = [
      int(line[indices[0]+1:].split(',')[0]),
      int(line[indices[1]+1:].split(':')[0]),
      int(line[indices[2]+1:].split(',')[0]),
      int(line[indices[3]+1:])]

  return sensor_x, sensor_y, beacon_x, beacon_y

## Part 1
We're only interest in one row, so build a set with all cells covered by manhattan distance.<br>
Return number of covered squares, not already covered by a beacon.

In [3]:
part1_y = 2000000
row = set()

for l in data:
  sensor_x, sensor_y, beacon_x, beacon_y = parse_line(l)
  man_dist = abs(sensor_x-beacon_x) + abs(sensor_y-beacon_y)

  if not sensor_y-man_dist <= part1_y <= sensor_y+man_dist:
    continue

  for x in range(sensor_x-(man_dist-abs(part1_y-sensor_y)), sensor_x+man_dist-abs(part1_y-sensor_y)+1):
    if beacon_x == x and beacon_y == part1_y:
      continue

    row.add(x)

print(len(row))

5125700


## Part 2 (incomplete)

In [6]:
rows = {}
magic_number = 4000000

skipped_rows = set()
full_row = set(range(magic_number+1))

for l in data:
  sensor_x, sensor_y, beacon_x, beacon_y = parse_line(l)
  man_dist = abs(sensor_x-beacon_x) + abs(sensor_y-beacon_y)

  for y in range(max(sensor_y-man_dist, 0), min(sensor_y+man_dist, magic_number)+1):
    if y in skipped_rows:
      continue

    if y not in rows:
      rows[y] = full_row.copy()

    for x in range(sensor_x-(man_dist-abs(y-sensor_y)), sensor_x+man_dist-abs(y-sensor_y)+1):
      if x in rows[y]:
        rows[y].remove(x)

    if len(rows[y]) == 0:
      skipped_rows.add(y)
      rows.pop(y, None)
        
print([i + list(r)[0] * 4000000 for (i, r) in rows.items()][0])